# TME 6 : Mini-projet Detection de motifs


## Recheche de pattern (motifs) en permettant des variations

Les motifs que nous cherchons dans les sequences d'ADN peuvent contenir quelques variations ou mutations. Nous allons developpé dans cet partie differents algoritmes pour la recherche de motifs variables. 

1\. Comme dans le TME5, nous alons d'abord générer des données atificielles pour pouvoir tester les algorithmes. Faites une fonction pour générer `t` séquences artificielles de taille `n`. Implantez dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.

In [195]:
import random

nuc = ('A', 'C', 'G', 'T')

k=4 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence

def generateRandomSequence(n, I):
    seq_tmp = ""
    
    if I == True:
        for i in range(n):
            seq_tmp = seq_tmp + random.choice(nuc)
    else:
        for i in range(n):
            seq_tmp = seq_tmp + random.choice(nuc).lower()

    return seq_tmp
    

def generateRandomSequences(n, t):
    sequences = [generateRandomSequence(n, True) for i in range(t)]
   
    return sequences


def implantMotifVar(k, v, t, n):
    sequences = generateRandomSequences(n, t)
    motif = generateRandomSequence(k, True)
    print("Motif : ", motif)
    
    for i in range(len(sequences)):
        L_seq = list(sequences[i])
        L_motif = list(motif)
        for nb_mutation in range(v):
            index_mutation_motif = random.randint(0,len(motif) - 1)
            L_motif[index_mutation_motif] = random.choice(nuc)
        chn_motif_mute = "".join(L_motif)
        print(chn_motif_mute)
        L_seq.insert(random.randint(0, len(L_seq) - 1), chn_motif_mute)
        
        sequences[i] = "".join(L_seq)
    return sequences

sequences = implantMotifVar(k, v, t, n)
print ("ADN : ", sequences)


Motif :  ACAG
ACAA
AGAG
AAAG
ADN :  ['TCACAATCTTTTTT', 'GGGGCCCAGAGGCT', 'TTAGCAAAAGACAG']


2\. Visualisation de motifs. Nous pouvons visualiser les motifs à l'aide des outils de LOGO https://weblogo.berkeley.edu/logo.cgi. Executer votre fonction de generation de motif variable, extraire les motifs et visulise-le à l'aide de webLogo. Utiliser les parametre ci-dessous.


In [196]:
k=8 #taille de motif
v=2 #nb de positions variable dans le motif
t=10 #nb de sequences
n=10 #longuer des sequence

sequences = implantMotifVar(k, v, t, n)
print (sequences)

Motif :  CCTATCAA
CCCACCAA
CCTATCTG
CTTATCAA
CTTATTAA
CCCATCAA
CCTATCCT
CCTAGCAA
CGTACCAA
GCTATCAA
ACTATAAA
['TAAGTCCCACCAATTCCG', 'GCAACATTCCTATCTGGA', 'GACTTATCAAAGATAACC', 'CCCTTATTAATCATATGC', 'CCCCATCAAAAGCGAGGT', 'CCCTATCCTATGAAAATA', 'GTCCCTAGCAACTTGATA', 'TCTTCGTACCAAGGCCGC', 'GAGCTATCAAGGCGCTCC', 'TCAGCTAGACTATAAATC']


3\. Préparation pour l'algorithme _"Brute Force Motif Search"_. Avant d'implementer  l'algorithme _"Brute Force Motif Search"_, faites une fonction pour calculer le score `score(s, seqs)`, où `s` est un ensemble de position initiales `s`$=(s_1,s_2,...,s_k)$  et `seqs` est une matrice de taille `t x n`  (`t` est le nombre de séquences et `n` la taille de ces séquences), pour tester votre fonction utiliser la matrice et les valeurs ci-dessous (voir aussi les slides du cours).


In [197]:
import numpy as np

k =8
n = 68 # length of each DNA sequence
t = 5 # number of  DNA sequence samples

s=(25,20,2,55,59)

seqs = [
'CCTGATAGACGCTATCTGGCTATCCAGGTACTTAGGTCCTCTGTGCGAATCTATGCGTTTCCAACCAT',
'AGTACTGGTGTACATTTGATCCATACGTACACCGGCAACCTGAAACAAACGCTCAGAACCAGAAGTGC',
'AAACGTTAGTGCACCCTCTTTCTTCGTGGCTCTGGCCAACGAGGGCTGATGTATAAGACGAAAATTTT',
'AGCCTCCGATGTAAGTCATAGCTGTAACTATTACCTGCCACCCCTATTACATCTTACGTCCATATACA',
'CTGTTATACAACGCGTCATGGCGGGGTATGCGTTTTGGTCGTCGTACGCTCGATCGTTACCGTACGGC'
]



#extract sequences from seqs
def extractSeqs(s, seqs):
    motifs = []
    
    L_zip = list(zip(s,seqs))
    
    for i_ini, seq in L_zip:
        seq_tmp = ""
        for i in range(i_ini, i_ini + k):
            seq_tmp = seq_tmp + seq[i]
        motifs.append(seq_tmp)
                
    return motifs


#construct Profiles
def profile(motifs, k, t, q):
    PWM = np.zeros((q, k), dtype = int)
    L = [x for x in nuc]
    
    dictionnaire = {}
    for i in range(len(L)):
        dictionnaire[L[i]] = PWM[i]
    
    for i in range(k):
        for j in range(len(motifs)):
            dictionnaire[motifs[j][i]][i] = dictionnaire[motifs[j][i]][i] + 1
            
    
    for i in range(q):
        PWM[i] = dictionnaire[L[i]]
        
    #print(PWM)
    
    return PWM


def score(s, seqs):
    sc = 0
    profile1 = profile(extractSeqs(s, seqs), k, t, len(nuc))
    
    L_max_colonne = []
    
    for i in range(len(profile1[0])):
        L_tmp = []
        for j in range(len(nuc)):
            L_tmp.append(profile1[j][i])
        sc = sc + max(L_tmp)
    
    return (sc)

sc = score(s, seqs)
#teste, sc doit etre egalle à 30
print (sc)



30


4\. Implementer l'algoritme _"Brute Force Motif Search"_ pour chercher des motifs de taille variable. Attention, la complexité de cet algorithme est trop importante, car il enumère toutes les positions intiales possibles, nous allons donc le tester seulement sur de petits jeux de données.

In [198]:
from itertools import product


#enumeration all initial positions
def initialPosition(k, n, t):
	iniPos = []; allIniPos = [];
	for i in range(0, n-k+1):
		iniPos.append(i)
        
	for subset in product(iniPos, repeat=t):
		allIniPos.append(subset)
	return allIniPos

def bruteForceMotifSearch(sequences, t, n, k):
    bestscore = 0
    bestMotif = ''#tuple des meilleurs position
    
    positionIni = initialPosition(k, n, t)
    
    for pos_tmp in positionIni:
        if score(pos_tmp, sequences) > bestscore:
            bestscore = score(pos_tmp, sequences)
            bestMotif = pos_tmp
            

    return bestMotif, bestscore

#test
k=4; v=1; t=3; n=10
sequences = implantMotifVar(k, v, t, n)
print (sequences)
bestMotif, bestscore = bruteForceMotifSearch(sequences, t, n, k)
print (bestMotif, bestscore)

Motif :  AGTG
CGTG
ACTG
AGTG
['CGTGTACAGTACTT', 'ACTGGCAAAGATTT', 'CGCAATATAGTGAC']
(3, 4, 1) 10


5\. L'algorithme est-il performant sur ce jeu de données ? Sinon, pourquoi ?

<font color='blue'>
réponse: On a k = 4, t = 3, n = 10, comme la complexité de cet algorithme est de O(kn^t), ici on fait donc
    4*10^3 soit 4000 calcul. Pour un ordinateur, ca reste encore faisable. Donc ici, comme c'est un petit jeu de donnée, cet algorithme reste "efficace"
</font>

6\. Tester l'agorithme avec les paramètres suivants `k=7`, `v=1`, `t=5`, et `n=20`. Avez vous de meilleurs performances ?

In [100]:
k=7; v=1; t=5; n=20

sequences = implantMotifVar(k, v, t, n)
print (sequences)
bestMotif, bestscore = bruteForceMotifSearch(sequences, t, n, k)

print (bestMotif)
#L'exécution ici a pris beaucoup plus de temps qu'avec les paramètres de la question 4.
#En effet ici on a 7 * 20^5 soit 22400000 calculs, ce qui prend quand même un peu de temps... même pour
#ordinateur ...

Motif :  GTTAGGA
GTTAGGA
GTTAGGT
GTGAGGA
GTTACGA
GTTAGGT
['GCTTGTGTGTGGTTGATGGGTTAGGAT', 'CTCTGTTAGGTACGCGGTGGTGTCTTG', 'GGGTAGAAGGGTGAGGATGTCAACTAG', 'GTTACGAGGTTAAGCATTTCTAGGATT', 'ACATAACCAGATTACGGGGTTAGGTGA']
(5, 10, 3, 2, 12)


7\. Implémentez l'algorithme _"Median String Search"_ pour chercher des motifs de taille variable. Tester le sur les mêmes jeux de données que la question 3 (voir slides de cours). Vous devez éliminer les motifs peu complexe pour éviter les calculs inutiles.

In [199]:
#generer les k-mers valid
k =8
n = 68 # length of each DNA sequence
t = 5 # number of  DNA sequence samples
minrep = 5

s=(25,20,2,55,59)

sequences = [
'CCTGATAGACGCTATCTGGCTATCCAGGTACTTAGGTCCTCTGTGCGAATCTATGCGTTTCCAACCAT',
'AGTACTGGTGTACATTTGATCCATACGTACACCGGCAACCTGAAACAAACGCTCAGAACCAGAAGTGC',
'AAACGTTAGTGCACCCTCTTTCTTCGTGGCTCTGGCCAACGAGGGCTGATGTATAAGACGAAAATTTT',
'AGCCTCCGATGTAAGTCATAGCTGTAACTATTACCTGCCACCCCTATTACATCTTACGTCCATATACA',
'CTGTTATACAACGCGTCATGGCGGGGTATGCGTTTTGGTCGTCGTACGCTCGATCGTTACCGTACGGC'
]
def tousLesKmers(ADN, k):
    """
    List(String) -> List(String)
    Rend la liste de tous les k-mers present dans chaque element de ADN
    """
    List_des_Kmer = []
    for sequence in ADN:
        for i in range(len(sequence)-k+1):
            seq_i = ""
            for j in range(k):
                seq_i = seq_i + sequence[i+j]
            List_des_Kmer.append(seq_i)
    return List_des_Kmer
            

def removeLowComplexe(motifs, minrep):
    validMotif = []
    for mot in motifs:
        if mot.count('A') < minrep and mot.count('C') < minrep and mot.count('G') < minrep and mot.count('T') < minrep:
            validMotif.append(mot)
    return validMotif

def removeLowComplexePair(motifs):
    validMotif = []
    
    for cle in motifs:
        list_tmp = []
        list_tmp.append(cle[0])
        for i in range(2, len(cle), 2):
            if cle[i] == list_tmp[0]:
                list_tmp.append(cle[i])
        cpt_pos_pair = len(list_tmp)
        list_tmp = []
        list_tmp.append(cle[1])
        for i in range(3, len(cle), 2):
            if cle[i] == list_tmp[0]:
                list_tmp.append(cle[i])
        cpt_pos_impair = len(list_tmp)
        
        if not cpt_pos_pair == ((len(cle) // 2) + 1) and not cpt_pos_impair == (len(cle) // 2):
            validMotif.append(cle)
              
    return validMotif
#Seulement avec sequences en majuscule ...

motifs_valides = tousLesKmers(sequences, k)
validkmers = removeLowComplexe(motifs_valides, minrep)
validkmers = removeLowComplexePair(validkmers)
print(validkmers)


['CCTGATAG', 'CTGATAGA', 'TGATAGAC', 'GATAGACG', 'ATAGACGC', 'TAGACGCT', 'AGACGCTA', 'GACGCTAT', 'ACGCTATC', 'CGCTATCT', 'GCTATCTG', 'CTATCTGG', 'TATCTGGC', 'ATCTGGCT', 'TCTGGCTA', 'CTGGCTAT', 'TGGCTATC', 'GGCTATCC', 'GCTATCCA', 'CTATCCAG', 'TATCCAGG', 'ATCCAGGT', 'TCCAGGTA', 'CCAGGTAC', 'CAGGTACT', 'AGGTACTT', 'GGTACTTA', 'GTACTTAG', 'TACTTAGG', 'ACTTAGGT', 'CTTAGGTC', 'TTAGGTCC', 'TAGGTCCT', 'AGGTCCTC', 'GGTCCTCT', 'GTCCTCTG', 'TCCTCTGT', 'CCTCTGTG', 'CTCTGTGC', 'TCTGTGCG', 'CTGTGCGA', 'TGTGCGAA', 'GTGCGAAT', 'TGCGAATC', 'GCGAATCT', 'CGAATCTA', 'GAATCTAT', 'AATCTATG', 'ATCTATGC', 'TCTATGCG', 'CTATGCGT', 'TATGCGTT', 'ATGCGTTT', 'TGCGTTTC', 'GCGTTTCC', 'CGTTTCCA', 'GTTTCCAA', 'TTTCCAAC', 'TTCCAACC', 'TCCAACCA', 'CCAACCAT', 'AGTACTGG', 'GTACTGGT', 'TACTGGTG', 'ACTGGTGT', 'CTGGTGTA', 'TGGTGTAC', 'GGTGTACA', 'GTGTACAT', 'TGTACATT', 'GTACATTT', 'TACATTTG', 'ACATTTGA', 'CATTTGAT', 'ATTTGATC', 'TTTGATCC', 'TTGATCCA', 'TGATCCAT', 'GATCCATA', 'ATCCATAC', 'TCCATACG', 'CCATACGT', 'CATACGTA', 'AT

In [200]:

def hamdist(v, x):
    diffs = 0
    for i in range(len(x)):
        if v[i] != x[i]:
            diffs = diffs + 1
    
    return diffs


def totalDistance(motif, sequences, k):
    td = 0
    
    for sequence_tmp in sequences:
        bestDistance = 1000
        for i in range(len(sequence_tmp)-k+1):
            seq_i = ""
            for j in range(k):
                seq_i = seq_i + sequence_tmp[i+j]
            if hamdist(motif, seq_i) < bestDistance:
                bestDistance = hamdist(motif, seq_i)
        
        td = td + bestDistance
        
    return td
#Ne marche qu'avec des sequence en MAJUSCULE ...


def MedianStringSearch(allkmers, sequences, t, n, k):
    bestDistance = 10000
    bestMotif = ""
    motifDist = {} #all motifs found
    
    for valid in allkmers:
        if totalDistance(valid, sequences, k) < bestDistance:
            bestDistance = totalDistance(valid, sequences, k)
            bestMotif = valid

    return bestMotif, bestDistance, motifDist

#teste, td = 4
#acgtacgt
td = totalDistance("ACGTACGT", sequences, k)
print (td)
bestMotif, bestDistance, motifDist = MedianStringSearch(validkmers, sequences, t, n, k)
print (bestMotif, bestDistance)

10
CTGATGTA 6


8\. Comparation entre les algorithmes _"Median String Search"_ et "_Brute Force Motif Search_"

a\. Produire des séquences artificielles avec les paramètres suivants  `k=7`, `v=1`, `t=5`, `n=20`. Comparer les performance et le temps de calculs des deux algorithmes.

In [201]:
k=7; v=1; t=5; n=20

sequences_question8 = implantMotifVar(k, v, t, n)
print(sequence_question8)

motifs_valides_question8 = tousLesKmers(sequences_question8, k)
validkmers_question8 = removeLowComplexe(motifs_valides_question8, minrep)
validkmers_question8 = removeLowComplexePair(validkmers_question8)
#print(validkmers_question8)



bestMotif_question8, bestDistance_question8, motifDist_question8 = MedianStringSearch(validkmers_question8, sequence_question8, t, n, k)
print (bestMotif_question8, bestDistance_question8)

#On remarque que ca prend beaucoup moins de temps que la methode _Brute Force Motif Search_

Motif :  CAAGTTC
CAAGTTC
CAAGTTC
CAAGTTC
CAAGTTC
AAAGTTC
['GAGGGTGGCCATCACGTATGAAGCAGG', 'ATAGGTCAATACCGGGAGCCCGCAAAG', 'GGGTGCAGATACTTGTCACCAGGATGC', 'CATGGGGTGGCAAGCCTAATTCAGACC', 'GGGGTGCCAAGGCTCAACGTTGCGAGT']
TGCAGAA 11


b\. Quel algorithme peut etre appliqué au probleme de recherche de motifs? Et comment?

<font color='blue'>
réponse: Le mieux est d'appliquer l'algorithme de "Median String Search" car il a moins de calculs à faire...
</font>

In [ ]:
k=7; n=80
minrep = 5

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    return sequence

genome = "Sequence_by_Peaks_8.fasta"


sequences_question9  = readFasta(genome, n)

t = len(sequences_question9)
print(t)


motifs_valides = tousLesKmers(sequences_question9, k)
validkmers_question9 = removeLowComplexe(motifs_valides, minrep)
validkmers_question9 = removeLowComplexePair(validkmers_question9)
#print(validkmers_question9)

bestMotif_question9, bestDistance_question9, motifDist_question9 = MedianStringSearch(validkmers_question9, sequences_question9, t, n, k)
print (bestMotif_question9, bestDistance_question9)

#resultat: C'est long à executer ...

912


9\. Ulitser la base YEASTRACT : http://www.yeastract.com/formsearchbydnamotif.php pour chercher les motifs. 
Avez vous une indication pour le facteur de transcription impliqué ? Comparer vos résultats aux résultats obtenu avec algorithme Table de Hachage.


<font color='blue'>
réponse: 
</font>